## Import Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")

## Define the Neural Network Architecture

We create a simple feed-forward network with:
- Input: 1 node (x value between 0 and 2π)
- Hidden layers: 64 → 64 → 32 nodes with ReLU activation
- Output: 1 node (predicted sin(x))

In [ ]:
class SineNet(nn.Module):
    """Simple neural network to approximate the sine function."""
    
    def __init__(self):
        super(SineNet, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize model
model = SineNet()
print("Model architecture:")
print(model)

## Generate Training Data

We generate 1000 evenly spaced points between 0 and 2π and compute their sine values.

In [ ]:
def generate_training_data(n_samples=1000):
    """Generates training and test data for the sine function."""
    x = np.linspace(0, 2 * np.pi, n_samples)
    y = np.sin(x)
    
    # Convert to PyTorch tensors
    x_tensor = torch.FloatTensor(x).reshape(-1, 1)
    y_tensor = torch.FloatTensor(y).reshape(-1, 1)
    
    return x_tensor, y_tensor

# Generate data
x_train, y_train = generate_training_data(n_samples=1000)
print(f"Training data shape: x={x_train.shape}, y={y_train.shape}")
print(f"x range: [{x_train.min():.2f}, {x_train.max():.2f}]")
print(f"y range: [{y_train.min():.2f}, {y_train.max():.2f}]")

## Visualize Training Data

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(x_train.numpy(), y_train.numpy(), linewidth=2)
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.title('Training Data: Sine Function')
plt.grid(True, alpha=0.3)
plt.show()

## Train the Model

We train the network using:
- Loss function: Mean Squared Error (MSE)
- Optimizer: Adam with learning rate 0.001
- Training epochs: 5000

In [ ]:
def train_model(model, x_train, y_train, epochs=5000, lr=0.001):
    """Trains the model."""
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    losses = []
    
    for epoch in range(epochs):
        # Forward pass
        predictions = model(x_train)
        loss = criterion(predictions, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        # Print progress
        if (epoch + 1) % 500 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}')
    
    return model, losses

# Train the model
print("Starting training...\n")
model, losses = train_model(model, x_train, y_train, epochs=5000, lr=0.001)
print(f"\nTraining complete! Final loss: {losses[-1]:.6f}")

## Visualize Training Loss

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(losses, linewidth=1)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss Over Time')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

## Evaluate the Model

Let's test the model on fresh data and compare predictions with actual sine values.

In [ ]:
# Generate test data
x_test, y_test = generate_training_data(n_samples=200)

# Evaluate
model.eval()
with torch.no_grad():
    predictions = model(x_test)

# Calculate MSE
mse = nn.MSELoss()(predictions, y_test)
print(f'Test MSE: {mse.item():.6f}')

# Calculate some sample predictions
test_points = torch.FloatTensor([[0.0], [np.pi/2], [np.pi], [3*np.pi/2], [2*np.pi]])
with torch.no_grad():
    sample_preds = model(test_points)

print("\nSample predictions:")
for x, pred in zip(test_points.numpy(), sample_preds.numpy()):
    actual = np.sin(x[0])
    print(f"x={x[0]:.4f}, predicted={pred[0]:.4f}, actual={actual:.4f}, error={abs(pred[0]-actual):.4f}")

## Visualize Results

Compare the actual sine function with the neural network's predictions.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(x_test.numpy(), y_test.numpy(), label='Actual Sine Function', linewidth=2, color='blue')
plt.plot(x_test.numpy(), predictions.numpy(), label='NN Prediction', linestyle='--', linewidth=2, color='red')
plt.xlabel('x', fontsize=12)
plt.ylabel('sin(x)', fontsize=12)
plt.title('Sine Approximation with Neural Network', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Error Analysis

Let's visualize the prediction error across the domain.

In [ ]:
errors = (predictions - y_test).numpy()

plt.figure(figsize=(12, 4))
plt.plot(x_test.numpy(), errors, linewidth=1.5, color='red')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xlabel('x', fontsize=12)
plt.ylabel('Prediction Error', fontsize=12)
plt.title('Prediction Error Across Domain', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Mean Absolute Error: {np.abs(errors).mean():.6f}")
print(f"Max Absolute Error: {np.abs(errors).max():.6f}")
print(f"Standard Deviation of Error: {errors.std():.6f}")